In [77]:
import gymnasium as gym
import numpy as np

In [78]:
env = gym.make('CartPole-v1')
print(env.observation_space.low, env.observation_space.high)

[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38] [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]


In [79]:
max_obs_values = np.array([2.4, 3, 0.2094 , 4])
min_obs_values = np.array([-2.4, -3, -0.2094, -4])
print(min_obs_values, max_obs_values)

[-2.4    -3.     -0.2094 -4.    ] [2.4    3.     0.2094 4.    ]


In [80]:
DISCRETE_OBS_SPACE_SIZE = [10]* len(max_obs_values) # these are the what the max values will corresponds to
discrete_obs_space_step_size = (max_obs_values - min_obs_values) / DISCRETE_OBS_SPACE_SIZE

def discretizer(obs):
    # obs = np.array([obs[0], obs[2]])
    discrete_obs = (obs - min_obs_values)/discrete_obs_space_step_size
    return tuple(discrete_obs.astype(np.int16)) # tuple to make indexing easier


In [81]:
LEARNING_RATE = 0.1
DISCOUNT = 0.95
EXPLORATION_RATE = 1
EXPLORATION_DECAY_RATE = 0.000001

In [82]:
q_table = np.zeros(DISCRETE_OBS_SPACE_SIZE + [env.action_space.n])
q_table.shape

(10, 10, 10, 10, 2)

In [83]:
print(q_table[0][0][0][0])
print(q_table[(0,0,0,0)]) 

[0. 0.]
[0. 0.]


In [84]:
observation, info = env.reset()
# obs = np.array([observation[0], observation[2]])
obs = observation
print(min_obs_values, max_obs_values)
print(obs)
discrete_obs = (obs - min_obs_values)/discrete_obs_space_step_size
discrete_obs = tuple(discrete_obs.astype(np.int16))
discrete_obs

[-2.4    -3.     -0.2094 -4.    ] [2.4    3.     0.2094 4.    ]
[-0.03024509  0.04589337 -0.0156145   0.00555891]


(4, 5, 4, 5)

In [111]:
num_episodes = 1000
average_scores = []
for e in range(num_episodes):
    observation, info = env.reset()
    current_obs = discretizer(observation) 
    done = False; score_per_episode = 0
    scores = [] # list for 100 episodes
    while not done:
        action = np.argmax(q_table[current_obs])
        if EXPLORATION_RATE > np.random.random():
             action = env.action_space.sample() 
        observation, reward, done, _, _ = env.step(action)
        discrete_obs = discretizer(observation)
        if not done:
            max_future_q = np.max(q_table[discrete_obs,:])
            current_q = q_table[current_obs + (action,)]
            new_q = (1-LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)
            q_table[current_obs + (action,)] = new_q
            # print(new_q, current_q) # new_q isn't changing
        else:
            print(discrete_obs, observation, done)
            

        current_obs = discrete_obs
        score_per_episode += reward
        EXPLORATION_RATE -= np.max(EXPLORATION_RATE - EXPLORATION_DECAY_RATE, 0)
    scores.append(score_per_episode)
        
    if e % 100 == 0:
        average_score = np.mean(scores)
        average_scores.append(average_score)
        scores = []
        print(f"Episode {e}, average score {average_score}")
    if score_per_episode > 10000:
        break
env.close()

# cart position is not really changing much for random actions, maybe i should use all the observations


(4, 1, 10, 8) [-0.20717488 -1.9313203   0.23249243  3.0635042 ] True
Episode 0, average score 10.0
(4, 2, 10, 8) [-0.14158359 -1.7959569   0.23685437  2.8509216 ] True
(4, 1, 10, 8) [-0.12465039 -1.9134161   0.24443118  3.0930884 ] True
(4, 2, 10, 8) [-0.15655214 -1.7848084   0.24638775  2.8112335 ] True
(4, 2, 10, 8) [-0.08399069 -1.523375    0.22088575  2.5399044 ] True
(4, 1, 10, 8) [-0.17213541 -1.917065    0.2336637   3.060905  ] True
(4, 1, 10, 8) [-0.13973223 -1.9185872   0.2308143   3.063857  ] True
(4, 2, 10, 8) [-0.13923989 -1.5790813   0.21943945  2.5866194 ] True
(4, 1, 10, 8) [-0.1792648  -1.980372    0.23895998  3.0236974 ] True
(4, 1, 10, 8) [-0.1340651  -1.9048388   0.21197213  2.969978  ] True
(4, 2, 10, 8) [-0.16335677 -1.7504144   0.22297603  2.827221  ] True
(4, 2, 10, 8) [-0.18395849 -1.7393295   0.23020132  2.8477602 ] True
(4, 1, 10, 8) [-0.16886838 -1.9561548   0.24360183  3.0612311 ] True
(4, 1, 10, 8) [-0.15873699 -1.9606001   0.22399233  3.045536  ] True
(4, 

IndexError: index 10 is out of bounds for axis 0 with size 10